In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd


train_df = pd.read_csv('/kaggle/input/training/train.csv',skiprows=0)
# test_df = pd.read_csv('/kaggle/input/testing/test.csv')
Y = train_df.filter(like="Blend")
X = train_df.drop(Y.columns,axis=1)

tf.keras.mixed_precision.set_global_policy('mixed_float16')


SHIFT_VALUE = 10
Y_shifted = Y + SHIFT_VALUE


2025-07-08 10:57:42.361830: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751972262.607726      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751972262.679383      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def feature_expansion(X):

    x_engg = X.copy()
    new_features = []
    
    for i in range(1,6):
        fractionc = f"Component{i}_fraction"
        proportionc = X[fractionc]
        propertyc  = f"Component{i}_Property"
        ComponentPropertiesc = X.filter(like=propertyc)  
        partial_componets = proportionc.values.reshape(-1, 1) * ComponentPropertiesc
    
        partial_componets.columns = [f'weighted_C{i}_P{j}' for j in range(1, 11)]
        
        new_features.append(partial_componets)
    
    X_final = pd.concat([x_engg] + new_features, axis=1 )
    return X_final

In [4]:
X_final = feature_expansion(X)
Y_target = Y

X_train, X_val, y_train_shifted, y_val_shifted = train_test_split(
    X_final, Y_shifted, test_size=0.2, random_state=42
)

BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train_shifted))
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val_shifted))
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10) 
])


optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='mean_absolute_error')
model.summary()

2025-07-08 10:57:58.319225: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        13,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,234 (94.66 KB)

 Trainable params: 24,234 (94.66 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
print("\nTraining the Neural Network on RAW data...")
history = model.fit(
    train_dataset,
    epochs=200, # Set a high number; early stopping will find the best epoch
    validation_data=val_dataset,
    callbacks=[early_stopping],
    verbose=1
)

print("\nEvaluating with the official (unstable) MAPE metric...")

val_predictions_shifted = model.predict(X_val)

val_predictions_original = val_predictions_shifted - SHIFT_VALUE
y_val_original = y_val_shifted - SHIFT_VALUE


mape_cost = mean_absolute_percentage_error(y_val_original, val_predictions_original)
print(f"Final Validation MAPE: {mape_cost:.4f}")

reference_cost = 2.72
leaderboard_score = 100 - (90 * mape_cost) / reference_cost
final_score = max(10, leaderboard_score)

print(f"\nYour BEST and MOST RELIABLE Estimated Public Leaderboard Score is: {final_score:.2f}")


Training the Neural Network on RAW data...
Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 9.8076 - val_loss: 8.4597
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.9640 - val_loss: 2.7038
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.0216 - val_loss: 1.2274
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1544 - val_loss: 1.1181
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9962 - val_loss: 1.0948
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9031 - val_loss: 1.0436
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.8378 - val_loss: 1.0282
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7883 - val_loss: 0.9948
Epoch 9/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7489 - val_loss: 0.9827
Epoch 10/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7114 - val_loss: 0.9698
Epoch 11/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6759 - val_loss: 0.9541
Epoch 12/200
25/25